In [1]:
!git clone https://github.com/NeuralFalconYT/FaceFinder.git
!pip install yt-dlp==2024.2.16.232705.dev0
!pip install ffmpeg==1.4
!pip install openvino-dev==2023.0.0
!pip install gradio
from IPython.display import clear_output
clear_output()

In [2]:
base_path = "/content/"

In [3]:
#@title Download YT Video
import re
import unicodedata
import os
import shutil

def clean_file_name(file_path):
    file_name, file_extension = os.path.splitext(os.path.basename(file_path))

    # Normalize the file name to remove emojis and special characters
    normalized_name = unicodedata.normalize('NFKD', file_name)

    # Remove emojis and unwanted characters using regex
    cleaned_name = re.sub(r'[\U00010000-\U0010FFFF]', '', normalized_name)  # Remove emojis
    cleaned_name = re.sub(r'[\[\](){}<>]', '', cleaned_name)  # Remove brackets
    cleaned_name = re.sub(r'[^\w\s-]', '', cleaned_name)  # Remove special characters except for underscores and hyphens
    cleaned_name = re.sub(r'\s+', ' ', cleaned_name).strip()  # Remove extra spaces
    cleaned_name = cleaned_name.replace(' ', '_')  # Replace spaces with underscores

    return cleaned_name[:25] + file_extension  # Limit filename length

def yt_video_download(yt_link):
    global base_path
    video_dir = os.path.join(base_path, "youtube_video")
    temp_download_dir = os.path.join(base_path, "temp_download")

    # Create necessary directories
    os.makedirs(video_dir, exist_ok=True)  # Create if not exists
    if os.path.exists(temp_download_dir):
        shutil.rmtree(temp_download_dir)  # Remove old temp folder
    os.makedirs(temp_download_dir)
    os.chdir(temp_download_dir)
    # Download the video
    var = os.system(f"yt-dlp {yt_link}")
    os.chdir(base_path)
    if var == 0:  # Check if the command was successful
        for file in os.listdir(temp_download_dir):
            if file.lower().endswith(('.mkv', '.mp4')):  # Check for video file extensions
                new_file_name = clean_file_name(file)
                shutil.copy(os.path.join(temp_download_dir, file), os.path.join(video_dir, new_file_name))
        return os.path.join(video_dir, new_file_name)  # Return the path of the last copied file
    return None  # Return None if download failed
YouTube_Video_Link="" # @param {type: "string"}
yt_vid_path=yt_video_download(YouTube_Video_Link)
yt_vid_path

'/content/youtube_video/Life_Inside_Brazilian_Meg.mp4'

In [27]:
#@title Enter video path
os.chdir(f"{base_path}/FaceFinder")

import os
import cv2
import time
from src.face_detector import FaceDetector
from src import utils
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm
# Change directory to the FaceFinder project folder
import importlib

# Reload the `utils` module from the `src` package
import subprocess
import re
import os
import zipfile
import uuid

def create_zip_from_folder():
    global base_path
    folder_path=f"{base_path}/FaceFinder/faces"
    # Generate a random UUID for the zip file name
    random_str=str(uuid.uuid4())[:6]
    zip_file_name = f"{random_str}.zip"
    save_zip_folder = os.path.join(base_path, "save_zip")

    # Create the save_zip directory if it doesn't exist
    os.makedirs(save_zip_folder, exist_ok=True)

    # Define the path for the zip file
    zip_file_path = os.path.join(save_zip_folder, zip_file_name)

    # Create a zip file and add the folder contents to it
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                # Add the file to the zip file, preserving the directory structure
                zip_file.write(file_path, os.path.relpath(file_path, folder_path))

    return zip_file_path

def video_duration(video_file):
    # Use ffprobe to get video file information
    command = ['ffprobe', '-v', 'error', '-show_entries', 'format=duration', '-of', 'default=noprint_wrappers=1:nokey=1', video_file]

    try:
        # Execute the command and capture the output
        duration = subprocess.check_output(command).decode().strip()

        # Convert duration to float and then to hours, minutes, seconds
        duration_seconds = float(duration)
        hours = int(duration_seconds // 3600)
        minutes = int((duration_seconds % 3600) // 60)
        seconds = int(duration_seconds % 60)

        # Print the formatted duration
        print(f"Video Duration: {hours}h {minutes}m {seconds}s")
    except subprocess.CalledProcessError:
        print("Error: Could not retrieve video duration.")
    except ValueError:
        print("Error: Invalid duration format.")

# Example usage



def main(video_source, confidence=0.5, skip_frames=False):
    importlib.reload(utils)
    global base_path
    if os.path.exists(f"{base_path}/FaceFinder/faces"):
      shutil.rmtree(f"{base_path}/FaceFinder/faces")
    os.makedirs(f"{base_path}/FaceFinder/faces")
    # Initialize the face detector with the specified model
    detector = FaceDetector(model=f'{base_path}/FaceFinder/model/public/ultra-lightweight-face-detection-rfb-320/FP16/ultra-lightweight-face-detection-rfb-320.xml',
                            confidence_thr=confidence,
                            overlap_thr=0.7)

    # Open the video source
    video = cv2.VideoCapture(video_source)

    # Automatically get FPS and total number of frames from the video source
    fps = video.get(cv2.CAP_PROP_FPS)
    number_of_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_skip = int(fps) if skip_frames else 1  # Set frame skip based on FPS or process every frame

    fps_cum = 0.0

    # Loop through the number of frames in the video
    for frame_number in tqdm(range(number_of_frames)):
        ret, frame = video.read()
        if not ret:
            print("Error reading frame from the video source.")
            break

        # Skip frames based on the frame skip setting
        if frame_number % frame_skip == 0:
            start_time = time.perf_counter()
            bboxes, scores = detector.inference(frame)
            end_time = time.perf_counter()

            fps = 1.0 / (end_time - start_time)
            fps_cum += fps
            fps_avg = fps_cum / ((frame_number // frame_skip) + 1)

            # Draw bounding boxes (you may want to save or process these frames further instead of displaying)
            frame = utils.draw_boxes_with_scores(frame, bboxes, scores)

            # Optionally save the frame or process it as needed
            # For example, you can save the processed frame:
            # cv2.imwrite(f'output/frame_{frame_number}.jpg', frame)

    video.release()
    cv2.destroyAllWindows()
video_path="video.mp4"  # @param {type: "string"}
video_duration(video_path)
main(video_source=video_path, confidence=0.5, skip_frames=True)

final_zip_path = create_zip_from_folder()
print(f"Zip file created at: {final_zip_path}")
from google.colab import files
files.download(final_zip_path)

#many bugs need to fix

In [31]:
# def find_face(video_path_or_link,confidence,skip_frames):
#   video_extensions = ('.mp4', '.mkv', '.avi', '.mov', '.flv', '.wmv', '.webm')
#   if not video_path_or_link.lower().endswith(video_extensions):
#     video_path=yt_video_download(video_path_or_link)
#   else:
#     video_path=video_path_or_link
#   confidence=confidence/100
#   video_duration(video_path)
#   main(video_source=video_path, confidence=0.5, skip_frames=skip_frames)
#   zip_path = create_zip_from_folder()
#   return zip_path

In [ ]:
# import gradio as gr
# # examples = [["https://www.youtube.com/afsdagdsgsda"]]
# gradio_inputs=[gr.Textbox(label="Enter a YouTube Link"),gr.Slider(50, 100, value=50, label="Detection Confidence"),gr.Checkbox(value=True,label="Skip Frames")]
# gradio_outputs=[gr.File(label="Download Facecs.ZIP")]
# gradio_interface = gr.Interface(fn=find_face, inputs=gradio_inputs,outputs=gradio_outputs , title="Extract Face From Video")#,examples=chatgpt_examples)
# gradio_interface.launch(debug=True)